In [16]:
# Time code execution
import time
import cython


In [91]:

# Import the needed modules
import numpy as np

# Define constants and given conditions
temp_arr = np.array([0.01, 0.1, 1, 2, 3, 4, 5, 10, 100])

# Define functions

# Create random X and Y that represents electrons initial
# orientation at +1 or -1
def create_electron_grid():
    electron_grid = np.random.randint(0, 1 + 1, (50, 50)) # Create the 50x50 grid
    electron_grid[electron_grid == 0] = -1 # Replace 0 with -1

    return electron_grid


def H_sigma(grid):
    start_time = time.perf_counter ()

    # Find the num of cols and rows of the grid array
    nrows, ncols = grid.shape

    # Create the variable containing the sigma value of the
    # below operation
    h_sigma = 0

    # Loop over all the rows of the grid array
    for i in range(nrows):
        # Loop over all the columns of the grid array
        for j in range(ncols):
            sigma_ij = grid[i, j] 
            sigma_i_minus_1_j = grid[i - 1, j] 
            
            # Check if i + 1 will overflow and replace it with 0
            if(i + 1 >= nrows):
                sigma_i_plus_1_j = grid[0, j]
            else:
                sigma_i_plus_1_j = grid[i + 1, j]

            sigma_i_j_minus_1 = grid[i, j - 1] 

            # Check if j + 1 will overflow and replace it with 0
            if(j + 1 >= ncols):
                sigma_i_j_plus_1 = grid[i, 0]
            else:
                sigma_i_j_plus_1 = grid[i, j + 1]
            
            # Calculate the sigma
            h_sigma += sigma_ij * (sigma_i_minus_1_j + sigma_i_plus_1_j + sigma_i_j_minus_1 + sigma_i_j_plus_1)
    
    # Multiply the -J/2 factor to get the final value and return it.
    h_sigma = h_sigma * (-1/2)
    # print("Time took to calculate energy", time.perf_counter () - start_time, "seconds")

    return h_sigma

# Flips one of the electron spin randomly
def flip_random_spin(grid):
    # Get the grid num of rows and columns
    nrows, ncols = grid.shape

    # Generate 2d random indices to flip electron's spin
    i, j = np.random.randint(0, nrows, grid.ndim)

    # Change the random electron's spin by multiply by -1
    grid[i, j] *= -1 

    # Return the new grid
    return grid, i, j

# Determine if the new energy of the electron grid should be
# accepted based on a probability that depends on the new energy.
def should_accept_new_energy(initial_energy, new_energy, temp):
    exponent = -1 * ((new_energy - initial_energy) / temp)
    probability = np.exp(exponent)

    # Generate a random number between [0, 1)
    rand_num = np.random.rand()

    if(rand_num <= probability):
        return True
    
    return False


# Calculates the average magnetic momentum of the given grid
def calc_avg_magnetic_momentum(grid):
    grid_sum = np.sum(grid)
    m = (1 / 600_000) * grid_sum
    return m


iterations = 100

# For each temperature, calculate the 600_000 iterations
for temp in temp_arr:
    # Calculate 5 independent measurements of average magnetic momentum
    for j in range (5):
        # Create a random grid
        grid = create_electron_grid()

        # Holds the 5 average magnetic momentum values
        m_arr = np.ones(5)

        # Calculate the initial energy
        initial_energy = H_sigma(grid)
            
        # Calculate the 600_000 iterations
        for i in range(iterations):
            # Flip a random 
            new_grid, row_idx, col_idx = flip_random_spin(grid)
            
            # Calculate the new energy
            new_energy = H_sigma(new_grid)

            # If the new energy is less than or equal to the previous
            # energy, we are going to accept the new grid and move on
            # to the next iteration.
            if(new_energy <= initial_energy):
                continue

            # If the energy is larger, then check based on the probability
            # to accept it.
            should_accept = should_accept_new_energy(initial_energy, new_energy, 1)

            # If the probability is true, accept it and move on to the next iteration
            if(should_accept == True):
                continue

            # If the program reaches this step, the new energy was not accepted,
            # revert back the grid
            grid[row_idx, col_idx] *= -1

        # Calculate the average magnetic momentum
        m = calc_avg_magnetic_momentum(grid)
        m_arr[j] = m
        print(max(m_arr))



1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
